In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import sklearn.metrics as sk
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import networkx as nx

C:\Users\Victor\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## Bases Iniciais

In [4]:
final =  pd.read_csv(
    'data/final.csv',
    sep=',',
    error_bad_lines=False
)

C:\Users\Victor\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (35,38,39,40,42,44,69,72,73,76,77,78,79,105) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Grafo

In [0]:
G = nx.DiGraph()
source = final['final_name_source']
target = final['final_name_target']

for k in range(0, len(source)):
  edge = (source[k], target[k])
  G.add_edge(*edge)

In [0]:
eigenvector_centrality = nx.eigenvector_centrality(G)
Id = []
centrality = []

for key in eigenvector_centrality:
  Id.append(key)
  centrality.append(eigenvector_centrality.get(key))

dic = {'Id': Id, 'centrality': centrality}
centrality_dataframe = pd.DataFrame(data=dic)

In [0]:
index = centrality_dataframe['Id'].to_list()
index = [x.lower() for x in index]

In [0]:
centrality_dataframe['Id'] = index
centrality_dataframe = centrality_dataframe.set_index('Id')
centrality_dataframe.head()

,centrality
Id,
lutjanus analis,0.000034
apogon maculatus,0.000696
lutjanus griseus,0.000218
cephalopholis cruentata,0.000627
aulostomus maculatus,0.000679


###Tratamento da base

In [0]:
classe = final[['final_name_source','Class_source']]
classe = classe.drop_duplicates()
classe = classe.set_index('final_name_source')
classe = classe.rename(columns={'final_name_source':'Id', 'Class_source':'classe'})
classe

,classe
final_name_source,
lutjanus analis,ACTINOPTERYGII
lutjanus griseus,ACTINOPTERYGII
cephalopholis cruentata,ACTINOPTERYGII
aulostomus maculatus,ACTINOPTERYGII
scorpaena grandicornis,ACTINOPTERYGII
...,...
zamenis longissimus,REPTILIA
cuora flavomarginata,REPTILIA
geochelone elegans,REPTILIA


In [0]:
population = final[['final_name_source','Population trend_source']]
population = population.drop_duplicates()
population = population.set_index('final_name_source')
population = population.rename(columns={'final_name_source':'Id', 'Population trend_source':'population'})
population

,population
final_name_source,
lutjanus analis,Decreasing
lutjanus griseus,Unknown
cephalopholis cruentata,Stable
aulostomus maculatus,Decreasing
scorpaena grandicornis,Unknown
...,...
zamenis longissimus,Unknown
cuora flavomarginata,NaN
geochelone elegans,Decreasing


In [0]:
red_list = final[['final_name_source','Red List threat category_source']]
red_list = red_list.drop_duplicates()
red_list = red_list.set_index('final_name_source')
red_list = red_list.rename(columns={'final_name_source':'Id', 'Red List threat category_source':'red_list'})
red_list

,red_list
final_name_source,
lutjanus analis,Near Threatened
lutjanus griseus,Least Concern
cephalopholis cruentata,Least Concern
aulostomus maculatus,Least Concern
scorpaena grandicornis,Least Concern
...,...
zamenis longissimus,Least Concern
cuora flavomarginata,Endangered
geochelone elegans,Vulnerable


In [0]:
classe = classe.loc[index]
classe = classe.dropna()

population = population.loc[index]
population = population.dropna()

red_list = red_list.loc[index]
red_list = red_list.dropna()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the do

In [0]:
realms = final[['final_name_source','Realms_source']]
realms = realms.drop_duplicates()
realms = realms.set_index('final_name_source')
realms = realms.rename(columns={'final_name_source':'Id', 'Realms_source':'realms'})

realms = realms.loc[index]
realms = realms.dropna()

realms_dummies = realms.copy()
lista = []

for t in realms_dummies.realms:
    t_aux = str(t).split('|')
    for i in t_aux:
        if i not in lista:
            lista.append(i)

help_dataframe = pd.DataFrame(0, index=realms_dummies.index, columns = lista)

for tam in lista:
    realms_dummies[tam] = realms_dummies['realms'].apply(lambda x: 1 if tam in str(x).split('|') else 0)

realms_dummies = realms_dummies.drop(columns=['realms'])
realms_dummies


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  


,Nearctic,Neotropical,Afrotropical,Australasian,Indomalayan,Palearctic,Oceanian,Antarctic
final_name_source,,,,,,,,
cephalopholis cruentata,1,1,0,0,0,0,0,0
aulostomus maculatus,1,1,0,0,0,0,0,0
sphyrna tiburo,1,1,0,0,0,0,0,0
carcharhinus acronotus,1,1,0,0,0,0,0,0
mustelus canis,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
zamenis longissimus,0,0,0,0,0,1,0,0
cuora flavomarginata,0,0,0,0,1,1,0,0
geochelone elegans,0,0,0,0,1,0,0,0


In [0]:
systems = final[['final_name_source','Systems_source']]
systems = systems.drop_duplicates()
systems = systems.set_index('final_name_source')
systems = systems.rename(columns={'final_name_source':'Id', 'Systems_source':'systems'})

systems = systems.loc[index]
systems = systems.dropna()

systems_dummies = systems.copy()

lista = []

for t in systems_dummies.systems:
    t_aux = str(t).split('|')
    for i in t_aux:
        if i not in lista:
            lista.append(i)

help_dataframe = pd.DataFrame(0, index=systems.index, columns = lista)

for tam in lista:
    systems_dummies[tam] = systems_dummies['systems'].apply(lambda x: 1 if tam in str(x).split('|') else 0)

systems_dummies = systems_dummies.rename(columns={'Freshwater (=Inland waters)': 'Freshwater'})
systems_dummies = systems_dummies.drop(columns=['systems'])
systems_dummies


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  


,Marine,Freshwater,Terrestrial
final_name_source,,,
lutjanus analis,1,0,0
lutjanus griseus,1,0,0
cephalopholis cruentata,1,0,0
aulostomus maculatus,1,0,0
scorpaena grandicornis,1,0,0
...,...,...,...
zamenis longissimus,0,0,1
cuora flavomarginata,0,1,1
geochelone elegans,0,0,1


## Teste com as classes dummificadas

In [0]:
qt_realms = pd.DataFrame()
qt_realms['qt_realms'] = realms_dummies.sum(axis = 1)

qt_systems = pd.DataFrame()
qt_systems['qt_systems'] = systems_dummies.sum(axis = 1)

todos = classe.join([qt_systems, systems_dummies, population, red_list, qt_realms, realms_dummies, centrality_dataframe])
todos = todos.dropna()
todos

,classe,qt_systems,Marine,Freshwater,Terrestrial,population,red_list,qt_realms,Nearctic,Neotropical,Afrotropical,Australasian,Indomalayan,Palearctic,Oceanian,Antarctic,centrality
final_name_source,,,,,,,,,,,,,,,,,
cephalopholis cruentata,ACTINOPTERYGII,1.0,1.0,0.0,0.0,Stable,Least Concern,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.265190e-04
aulostomus maculatus,ACTINOPTERYGII,1.0,1.0,0.0,0.0,Decreasing,Least Concern,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.787715e-04
sphyrna tiburo,CHONDRICHTHYES,1.0,1.0,0.0,0.0,Stable,Least Concern,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,7.901115e-05
carcharhinus acronotus,CHONDRICHTHYES,1.0,1.0,0.0,0.0,Decreasing,Near Threatened,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,7.901115e-05
mustelus canis,CHONDRICHTHYES,1.0,1.0,0.0,0.0,Unknown,Near Threatened,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.248606e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
oxyuranus scutellatus,REPTILIA,1.0,0.0,0.0,1.0,Stable,Least Concern,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.556661e-26
zamenis longissimus,REPTILIA,1.0,0.0,0.0,1.0,Unknown,Least Concern,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.556661e-26
geochelone elegans,REPTILIA,1.0,0.0,0.0,1.0,Decreasing,Vulnerable,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.080694e-21


## Teste sem as classes dummificadas

In [0]:
model = ols("centrality ~ C(classe) + C(population) + C(red_list) + qt_systems + qt_realms + Terrestrial + Freshwater + Marine + Afrotropical + Australasian + Indomalayan + Nearctic + Neotropical + Palearctic + Oceanian + Antarctic",  data=todos)
#model = ols("eigencentrality ~ C(classe) + C(red_list)", data=todos)
response = model.fit()
response.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             centrality   R-squared:                       0.181
Model:                            OLS   Adj. R-squared:                  0.172
Method:                 Least Squares   F-statistic:                     20.20
Date:                Tue, 26 Nov 2019   Prob (F-statistic):           2.21e-84
Time:                        02:15:39   Log-Likelihood:                 6365.0
No. Observations:                2396   AIC:                        -1.268e+04
Df Residuals:                    2369   BIC:                        -1.252e+04
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                          -0.0043      0.003     -1.684      0.092      -0.009       0.001
C(classe)[T.AMPHIBIA]              -0.0087      0.003     -3.006      0.003      -0.014      -0.003
C(classe)[T.AVES]                  -0.0069      0.003     -2.688      0.007      -0.012      -0.002
C(classe)[T.CEPHALASPIDOMORPHI]    -0.0077      0.007     -1.179      0.238      -0.021       0.005
C(classe)[T.CHONDRICHTHYES]         0.0012      0.002      0.592      0.554      -0.003       0.005
C(classe)[T.MAMMALIA]              -0.0008      0.002     -0.332      0.740      -0.005       0.004
C(classe)[T.REPTILIA]              -0.0104      0.003     -3.882      0.000      -0.016      -0.005
C(classe)[T.SARCOPTERYGII]          0.0050      0.012      0.412      0.681      -0.019       0.029
C(population)[T.Increasing]         0.0057      0.001      4.210      0.000       0.003       0.008
C(population)[T.Stable]             0.0008      0.001      0.748      0.455      -0.001       0.003
C(population)[T.Unknown]           -0.0019      0.001     -1.627      0.104      -0.004       0.000
C(red_list)[T.Data Deficient]       0.0018      0.003      0.573      0.567      -0.004       0.008
C(red_list)[T.Endangered]          -0.0023      0.003     -0.845      0.398      -0.008       0.003
C(red_list)[T.Least Concern]        0.0026      0.002      1.087      0.277      -0.002       0.007
C(red_list)[T.Near Threatened]     -0.0017      0.003     -0.641      0.521      -0.007       0.003
C(red_list)[T.Vulnerable]          -0.0012      0.003     -0.447      0.655      -0.006       0.004
qt_systems                          0.0017      0.001      2.324      0.020       0.000       0.003
qt_realms                           0.0021      0.000      6.677      0.000       0.002       0.003
Terrestrial                         0.0080      0.002      4.656      0.000       0.005       0.011
Freshwater                         -0.0007      0.001     -0.698      0.485      -0.003       0.001
Marine                             -0.0056      0.001     -6.169      0.000      -0.007      -0.004
Afrotropical                       -0.0021      0.001     -2.376      0.018      -0.004      -0.000
Australasian                       -0.0016      0.001     -1.301      0.193      -0.004       0.001
Indomalayan                        -0.0008      0.001     -0.721      0.471      -0.003       0.001
Nearctic                            0.0106      0.001     14.029      0.000       0.009       0.012
Neotropical                        -0.0019      0.001     -2.265      0.024      -0.003      -0.000
Palearctic                         -0.0010      0.001     -1.092      0.275      -0.003       0.001
Oceanian                           -0.0015      0.001     -1.037      0.300      -0

###Teste com as classes não dummificadas

In [0]:
todos = classe.join([qt_systems, systems, population, red_list, qt_realms, realms, centrality_dataframe])
todos = todos.dropna()
todos

,classe,qt_systems,systems,population,red_list,qt_realms,realms,centrality
final_name_source,,,,,,,,
cephalopholis cruentata,ACTINOPTERYGII,1.0,Marine,Stable,Least Concern,2.0,Nearctic|Neotropical,6.265190e-04
aulostomus maculatus,ACTINOPTERYGII,1.0,Marine,Decreasing,Least Concern,2.0,Nearctic|Neotropical,6.787715e-04
sphyrna tiburo,CHONDRICHTHYES,1.0,Marine,Stable,Least Concern,2.0,Nearctic|Neotropical,7.901115e-05
carcharhinus acronotus,CHONDRICHTHYES,1.0,Marine,Decreasing,Near Threatened,2.0,Nearctic|Neotropical,7.901115e-05
mustelus canis,CHONDRICHTHYES,1.0,Marine,Unknown,Near Threatened,2.0,Nearctic|Neotropical,1.248606e-02
...,...,...,...,...,...,...,...,...
oxyuranus scutellatus,REPTILIA,1.0,Terrestrial,Stable,Least Concern,1.0,Australasian,1.556661e-26
zamenis longissimus,REPTILIA,1.0,Terrestrial,Unknown,Least Concern,1.0,Palearctic,1.556661e-26
geochelone elegans,REPTILIA,1.0,Terrestrial,Decreasing,Vulnerable,1.0,Indomalayan,4.080694e-21


In [0]:
model = ols("centrality ~ C(classe) + C(population) + C(red_list) + C(systems) + C(realms) + qt_systems + qt_realms",  data=todos)
response = model.fit()
response.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             centrality   R-squared:                       0.231
Model:                            OLS   Adj. R-squared:                  0.192
Method:                 Least Squares   F-statistic:                     5.899
Date:                Tue, 26 Nov 2019   Prob (F-statistic):           1.28e-68
Time:                        02:22:26   Log-Likelihood:                 6439.6
No. Observations:                2396   AIC:                        -1.265e+04
Df Residuals:                    2279   BIC:                        -1.197e+04
Df Model:                         116                                         
Covariance Type:            nonrobust                                         
=========================================================================================================================================================================
                                                                                                            coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                              8.509e+09   2.52e+10      0.337      0.736    -4.1e+10     5.8e+10
C(classe)[T.AMPHIBIA]                                                                                    -0.0048      0.003     -1.490      0.136      -0.011       0.002
C(classe)[T.AVES]                                                                                        -0.0042      0.003     -1.492      0.136      -0.010       0.001
C(classe)[T.CEPHALASPIDOMORPHI]                                                                          -0.0080      0.007     -1.222      0.222      -0.021       0.005
C(classe)[T.CHONDRICHTHYES]                                                                               0.0039      0.002      1.636      0.102      -0.001       0.009
C(classe)[T.MAMMALIA]                                                                                     0.0022      0.003      0.818      0.413      -0.003       0.008
C(classe)[T.REPTILIA]                                                                                    -0.0075      0.003     -2.551      0.011      -0.013      -0.002
C(classe)[T.SARCOPTERYGII]                                                                                0.0007      0.014      0.050      0.960      -0.027       0.028
C(population)[T.Increasing]                                                                               0.0057      0.001      4.188      0.000       0.003       0.008
C(population)[T.Stable]                                                                                   0.0006      0.001      0.565      0.572      -0.001       0.003
C(population)[T.Unknown]                                                                                 -0.0017      0.001     -1.473      0.141      -0.004       0.001
C(red_list)[T.Data Deficient]                                                                             0.0021      0.003      0.652      0.514      -0.004       0.009
C(red_list)[T.Endangered]                                                                                -0.0017      0.003     -0.617      0.538      -0.007       0.004
C(red_list)[T.Least Concern]                                                                              0.0029      0.002      1.194      0.233      -0.002       0.008
C(red_list)[T.Near Threatened]                                                                           -0.0018      0.003     -0.684      0.494      -0.007       0.003
C(red_list)[T.Vulnerabl